In [1]:
import pandas as pd
import ast
from data_process import MLPDataset_For_League, make_champ_idx
from torch.utils.data import DataLoader
from BERT import BERT,BERTLM
# from WinnerPredictModel import Winner_Predictor, Winner_Predictor_Trainer
from CNN import CNN, CNN_Trainer
import torch

In [2]:
make_champ_idx("../Data/match_data_2.csv", 5)

{1: 5,
 2: 6,
 3: 7,
 4: 8,
 5: 9,
 6: 10,
 7: 11,
 8: 12,
 9: 13,
 10: 14,
 11: 15,
 12: 16,
 13: 17,
 14: 18,
 15: 19,
 16: 20,
 17: 21,
 18: 22,
 19: 23,
 20: 24,
 21: 25,
 22: 26,
 23: 27,
 24: 28,
 25: 29,
 26: 30,
 27: 31,
 28: 32,
 29: 33,
 30: 34,
 31: 35,
 32: 36,
 33: 37,
 34: 38,
 35: 39,
 36: 40,
 37: 41,
 38: 42,
 39: 43,
 40: 44,
 41: 45,
 42: 46,
 43: 47,
 44: 48,
 45: 49,
 48: 50,
 50: 51,
 51: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 67: 65,
 68: 66,
 69: 67,
 72: 68,
 74: 69,
 75: 70,
 76: 71,
 77: 72,
 78: 73,
 79: 74,
 80: 75,
 81: 76,
 82: 77,
 83: 78,
 84: 79,
 85: 80,
 86: 81,
 89: 82,
 90: 83,
 91: 84,
 92: 85,
 96: 86,
 98: 87,
 99: 88,
 101: 89,
 102: 90,
 103: 91,
 104: 92,
 105: 93,
 106: 94,
 107: 95,
 110: 96,
 111: 97,
 112: 98,
 113: 99,
 114: 100,
 115: 101,
 117: 102,
 119: 103,
 120: 104,
 121: 105,
 122: 106,
 126: 107,
 127: 108,
 131: 109,
 133: 110,
 134: 111,
 136: 112,
 141:

In [28]:
df = pd.read_csv("../Data/match_data_validation.csv")
df["teams"] = df["teams"].apply(lambda x : ast.literal_eval(x))


champ_idx = {}
with open("champ_idx.txt", 'r') as file:
    for line in file:
        pair = ast.literal_eval(line)
        champ_idx.update({pair[0]: pair[1]})



for i in range(len(df["teams"])):
    for j in range(len(df["teams"][i])):
        df["teams"][i][j] = champ_idx[df["teams"][i][j]]


train_datas = [[df["teams"][i][:5], df["teams"][i][5:], df["winner"][i]] for i in range(len(df))]

In [29]:
MAX_LEN = 13
vocab_size = 171

train_data = MLPDataset_For_League(
train_datas, seq_len=MAX_LEN)

test_datas = {"bert_input": [], "segment_label": [], "winner_label": []}
for data in train_data:
    for idx, d in data.items():
        test_datas[idx].append(d)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_datas = {key: torch.stack(value, dim = 0).to(device) for key, value in test_datas.items()}


# train_loader = DataLoader(
# train_data, batch_size=16, shuffle=True, pin_memory=True)


In [37]:
bert_model = BERT(
vocab_size=vocab_size,
d_model= 32,
n_layers=8,
heads=8,
dropout=0.1,
device = device
)

bert_ = BERTLM(bert_model, vocab_size)
bert_.load_state_dict(torch.load("../Trained_Model/bert_model_5"))


cn = CNN(bert_).to(device)
cn.load_state_dict(torch.load("../Trained_Model/cnn_model4"))

cn.eval()

winner_output = cn.forward(test_datas["bert_input"], test_datas["segment_label"])

# correct = winner_output.argmax(dim=-1).eq(test_datas["winner_label"]).sum().item()
correct = torch.round(torch.flatten(winner_output)).eq(test_datas["winner_label"]).sum().item()
total = correct/len(winner_output)
total

0.511

In [59]:

wpm = Winner_Predictor(bert_).to(device)
wpm.load_state_dict(torch.load("../Trained_Model/mlp_model"))

wpm.eval()

winner_output = wpm.forward(test_datas["bert_input"], test_datas["segment_label"])

# correct = winner_output.argmax(dim=-1).eq(test_datas["winner_label"]).sum().item()
correct = torch.round(torch.flatten(winner_output)).eq(test_datas["winner_label"]).sum().item()
total = correct/len(winner_output)
total

0.5086666666666667

In [5]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

bert_.eval()
x = (bert_.embedding(test_datas["bert_input"], test_datas["segment_label"])).to("cpu")
embedded_x = torch.cat((x[:, 1:6], x[:, 7:-1]), dim = 1)
embedded_x = torch.flatten(embedded_x, start_dim=1)

from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(embedded_x, test_datas["winner_label"].to("cpu"))


# Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('svc', SVC(gamma='auto'))])


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [8]:
x = (bert_.embedding(test_datas["bert_input"], test_datas["segment_label"])).to("cpu")
embedded_x = torch.cat((x[:, 1:6], x[:, 7:-1]), dim = 1)
embedded_x = torch.flatten(embedded_x, start_dim=1)
clf.score(embedded_x, test_datas["winner_label"].to("cpu"))


0.5193333333333333

In [7]:
id = 111
print(winner_output[id], test_datas["winner_label"][id])

tensor([0.5836], device='cuda:0', grad_fn=<SelectBackward0>) tensor(0, device='cuda:0')
